In [1]:
# Import the dependencies
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=1500)
lngs = np.random.uniform(-90, 90, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

613

In [21]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind})

        #city_data.append({"City": city.title(),
                         # "Lat": city_lat,
                          #"Lng": city_lng,
                          #"Max Temp": city_max_temp,
                          #"Humidity": city_humidity,
                          #"Cloudiness": city_clouds,
                          #"Wind Speed": city_wind,
                          #"Country": city_country,
                          #"Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jamestown
Processing Record 2 of Set 1 | narsaq
Processing Record 3 of Set 1 | havre-saint-pierre
Processing Record 4 of Set 1 | zhob
Processing Record 5 of Set 1 | souillac
Processing Record 6 of Set 1 | cidreira
Processing Record 7 of Set 1 | bolshegrivskoye
City not found. Skipping...
Processing Record 8 of Set 1 | kassala
Processing Record 9 of Set 1 | abha
Processing Record 10 of Set 1 | kruisfontein
Processing Record 11 of Set 1 | penzance
Processing Record 12 of Set 1 | dikson
Processing Record 13 of Set 1 | adrar
Processing Record 14 of Set 1 | longyearbyen
Processing Record 15 of Set 1 | goderich
Processing Record 16 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | ndjole
Processing Record 19 of Set 1 | havelock
Processing Record 20 of Set 1 | farafangana
Processing Record 21 of Set 1 | hermanus
Processing Reco

In [22]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

# Fixed above
#new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
#city_data_df = city_data_df.reindex(columns=new_column_order)


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Jamestown,US,2021-09-30 22:26:01,42.0970,-79.2353,58.32,70,75,4.00
1,Narsaq,GL,2021-09-30 22:30:00,60.9167,-46.0500,34.02,48,0,2.91
2,Havre-Saint-Pierre,CA,2021-09-30 22:30:00,50.2334,-63.5986,44.60,93,90,16.11
3,Zhob,PK,2021-09-30 22:30:01,31.3411,69.4481,74.28,60,0,4.12
4,Souillac,MU,2021-09-30 22:30:01,-20.5167,57.5167,72.36,78,75,13.80
5,Cidreira,BR,2021-09-30 22:30:03,-30.1811,-50.2056,65.46,79,100,26.11
6,Kassala,SD,2021-09-30 22:30:04,15.4510,36.4000,83.93,50,53,2.53
7,Abha,SA,2021-09-30 22:30:05,18.2164,42.5053,64.65,37,0,4.61
8,Kruisfontein,ZA,2021-09-30 22:30:06,-34.0033,24.7314,58.89,97,100,2.53
9,Penzance,GB,2021-09-30 22:30:07,50.1186,-5.5371,59.68,90,100,21.85


In [23]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")